# Spatially Informed Traveling Salesman Problem
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Rw9_fomI2CRgYNwEmZRx8XU3j1RQK9gO)
Developed by Wanhee Kim(Phd student at UTK) / Revised Date : 04/09/2024


## Sequence of spatially informed TSP
0. Pre-setting for the Analysis
1. Generate distance matrix regarding on Spatial adjacency
2. Generate LP Model for Analyzing TSP
3. Analyze by using CPLEX(Solving machine)

## Analysis

0. Pre-Setting for Analysis

In [ ]:
pip install spatialtsp

In [ ]:
# change path
import os
os.chdir('D:/GIS program/Github/spatialtsp/spatialtsp-5')

# Check the directory
# current_directory = os.getcwd()
# print("current directory:", current_directory)

# import spatialtsp
# print(dir(spatialtsp))

# # set working directory
# path = 'D:/GIS program/Github/spatialtsp/final_work'
# os.chdir(path)
# os.getcwd()

In [ ]:
# Import the package
from spatialtsp import generate_clustered_points, generate_random_points, calculate_distance_matrix, voronoi_adjacency_distance, knn_adjacency_distance, combine_distance_matrices, generate_lp_model, get_attributes_cplex, writeLpFile_func

In [ ]:
#print(dir(spatialtsp))

## Test in Toy Data

1. Generate Distance Matrix(standard vs Spatially Informed)

In [ ]:
# 1) Generate points
points=generate_random_points(7) # Generate Stratified random points. 7*7 = 49

#print(points)

In [ ]:
# 2) Calculate Standard Distance Matrix
distance_matrix=calculate_distance_matrix(points)
distance_matrix

In [ ]:
# 3) Calculate Distance Matrix with the concept of adjacency : by using Voronoi concept and K-NN concept
Voronoi_distance, voronoi_polygon = voronoi_adjacency_distance(points)
KNN_distance = knn_adjacency_distance(points, k=7)
combined_distance = combine_distance_matrices(Voronoi_distance, KNN_distance) # Combine the two distance matrices

# Print the distance matrices
print(Voronoi_distance)
print(KNN_distance)
print(combined_distance)

In [ ]:
# Export shp files
path = 'D:/GIS program/Github/spatialtsp/spatialtsp-5'
points.to_file('{}/final_work/03_LPFiles/test.shp'.format(path), encoding='utf-8')
voronoi_polygon.to_file('{}/final_work/03_LPFiles/test_polygon.shp'.format(path), encoding='utf-8')

### 2. Generate LP Model

2-1. Make LP files

In [ ]:
# Standard version
lp_model = generate_lp_model(distance_matrix)
# Spatially Informed version
lp_model_2 = generate_lp_model(combined_distance)

print(lp_model)
print(lp_model_2)

2-2. Write the LP file

In [ ]:
# Standard version
file_path = f"{path}/final_work/03_LPFiles/TSP_test.lp" 
with open(file_path, 'w') as file:
    file.write(lp_model)
print(f"LP file saved to {file_path}")

# Spatially Informed version
file_path = f"{path}/final_work/03_LPFiles/TSP_test_2.lp" 
with open(file_path, 'w') as file:
    file.write(lp_model_2)
print(f"LP file saved to {file_path}")

### 3. Run LP Model

In [ ]:
# Import the package
import subprocess
import pandas as pd

Standard TSP

In [ ]:
# Define the path to the LP file
lp_file_path = f'"{path}/final_work/03_LPFiles/TSP_test.lp"'

# Initialize lists to store results
objval_ls = []
nodenb_ls = []
timenb_ls = []
iternb_ls = []
dettime_ls = []

# Run CPLEX
command = [
    'D:/GIS program/cplex_2018-20230804T132516Z-003/cplex/bin/x64_win64/cplex.exe', '-c',
    f'read {lp_file_path}',
    'set threads 2',
    'set timelimit 3600',
    'opt', 
    'display solution variables -'
]
try:
    r = subprocess.run(command, capture_output=True, text=True, check=True)
    print("STDOUT:", r.stdout)
    print("STDERR:", r.stderr)
except subprocess.CalledProcessError as e:
    print("Error:", e)
    print("STDERR:", e.stderr)

# Get the attributes by running CPLEX
result = r.stdout
timenb, iternb, nodenb, objval, dettime, variables = get_attributes_cplex(result)
print(f"Objective: {objval}, Solution Time: {timenb}, Iterations: {iternb}, Nodes: {nodenb}, Dettime: {dettime}")
print("Variables:", variables)

# Append the results to the lists
objval_ls.append(objval)  
timenb_ls.append(timenb)  
iternb_ls.append(iternb)  
nodenb_ls.append(nodenb) 
dettime_ls.append(dettime) 

loc_dict = {
    'ObjVal': objval_ls, 
    'Nodenb': nodenb_ls, 
    'Timenb': timenb_ls, 
    'Iternb': iternb_ls, 
    'Dettime': dettime_ls,
    'Variables': [variables]
     #'Geogetry' : geometry_ls

}
df_1 = pd.DataFrame(loc_dict)

df_1

Spatially Informed TSP

In [ ]:
import subprocess
import pandas as pd

lp_file_path_2 = f'"{path}/final_work/03_LPFiles/TSP_test_2.lp"'

# Initialize lists to store results
objval_ls = []
nodenb_ls = []
timenb_ls = []
iternb_ls = []
dettime_ls = []

# Run CPLEX
command = [
    'D:/GIS program/cplex_2018-20230804T132516Z-003/cplex/bin/x64_win64/cplex.exe', '-c',
    f'read {lp_file_path_2}',
    'set threads 2',
    'set timelimit 3600',
    'opt',
    'display solution variables -'
]
try:
    r = subprocess.run(command, capture_output=True, text=True, check=True)
    print("STDOUT:", r.stdout)
    print("STDERR:", r.stderr)
except subprocess.CalledProcessError as e:
    print("Error:", e)
    print("STDERR:", e.stderr)

# Get the attributes by running CPLEX
result = r.stdout
timenb, iternb, nodenb, objval, dettime, variables = get_attributes_cplex(result)
print(f"Objective: {objval}, Solution Time: {timenb}, Iterations: {iternb}, Nodes: {nodenb}, Dettime: {dettime}")
print("Variables:", variables)

# Append the results to the lists
objval_ls.append(objval)  
timenb_ls.append(timenb)  
iternb_ls.append(iternb)  
nodenb_ls.append(nodenb) 
dettime_ls.append(dettime) 

loc_dict = {
    'ObjVal': objval_ls, 
    'Nodenb': nodenb_ls, 
    'Timenb': timenb_ls, 
    'Iternb': iternb_ls, 
    'Dettime': dettime_ls,
    'Variables': [variables]
     #'Geogetry' : geometry_ls
}
df_2 = pd.DataFrame(loc_dict)

df_2

Visualize the results

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import LineString

In [ ]:
# Extract the order of tour
# 'variables' means the order of the nodes for tour
tour_order = [(int(k.split('_')[1]), int(k.split('_')[2])) for k, v in variables.items() if v == 1.0]
print(tour_order)

In [ ]:
# Read the shapefiles(points and polygons) for visualization
polygon_path = "D:\\GIS program\\Github\\spatialtsp\\spatialtsp-5\\final_work\\03_LPFiles\\test_polygon.shp"
point_path = "D:\\GIS program\\Github\\spatialtsp\\spatialtsp-5\\final_work\\03_LPFiles\\test.shp"
gdf_points = gpd.read_file(point_path)
gdf_polygons = gpd.read_file(polygon_path)

In [ ]:
def find_point(fid):
    try:
        point = gdf_points[gdf_points['FID'] == fid-1].geometry.iloc[0]
        return point
    except IndexError:
        raise ValueError(f"No geometry found for FID {fid}. Check if the FID is correct and present in gdf_points.")

# Generate lines for the tour
lines = [LineString([find_point(start), find_point(end)]) for start, end in tour_order]

# # Create a GeoDataFrame for the lines
gdf_lines = gpd.GeoDataFrame(geometry=lines, crs=gdf_points.crs)

# Plot
fig, ax = plt.subplots(figsize=(12, 10))
gdf_polygons.plot(ax=ax, color='lightgrey', edgecolor='black', alpha=0.1)
gdf_points.plot(ax=ax, marker='o', color='blue', markersize=5)
gdf_lines.plot(ax=ax, linewidth=1.5, color='red')

# Adding labels
for Input_FID, row in gdf_points.iterrows():
    ax.annotate(text=Input_FID+1, xy=(row.geometry.x, row.geometry.y),
                xytext=(3, 3), textcoords="offset points", color='black')

ax.set_title('TSP Solution')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.show()

### Application into 48 Capitals of US

In [ ]:
# Import the package
import numpy as np
import geopandas as gpd

In [ ]:
# Read the shapefiles
# polygon_path = "C:\\Users\\dooco\\OneDrive\\spatial data\\US_Data\\TSP_dataset\\SpatialData\\Capitals_Voronoi_clipped.shp"
# point_path = "C:\\Users\\dooco\\OneDrive\\spatial data\\US_Data\\TSP_dataset\\SpatialData\\Capitals_Points.shp"

# US_points = gpd.read_file(point_path)
# #US_points = np.array(list(gdf_points.geometry.apply(lambda geom: (geom.x, geom.y))))

In [ ]:
# Calculate the distance matrix
# distance_matrix=calculate_distance_matrix(US_points)
# distance_matrix

In [ ]:
# Calculate the distance matrix with the concept of adjacency
# Voronoi_distance = voronoi_adjacency_distance(US_points)
# KNN_distance = knn_adjacency_distance(US_points, k=7)
# combined_distance = combine_distance_matrices(Voronoi_distance, KNN_distance)
# combined_distance

In [ ]:
# set working directory
# import os

# path = 'D:/GIS_analyzing/1.Standard_TSP/0.test_iteration' # write your own directory
# os.chdir(path)
# os.getcwd()

In [ ]:
## other codes will be updated soon

## Display Basemap

In [ ]:
## Add basemap
from spatialtsp import Map

my_map = Map(center=[40.7128, -74.0060], zoom=10)
my_map.add_basemap("OpenStreetMap.Mapnik")

my_map


In [ ]:
## Add GeoJSON
from spatialtsp import Map

geojson_map = Map(center=[37.0902, -95.7129], zoom=4)
geojson_url = "https://github.com/opengeos/datasets/releases/download/us/us_states.geojson"
geojson_map.add_geojson(geojson_url, name="US Counties")

geojson_map


In [ ]:
## Add shp
import geopandas as gpd
import requests
import zipfile
import io
from spatialtsp import Map

# Shapefile URL
url = 'https://github.com/opengeos/datasets/releases/download/us/us_states.zip'

# Download and extract the zip file
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))

# Find the shapefile name
shp_name = [name for name in z.namelist() if '.shp' in name][0]

# Read and extract the shapefile
z.extractall("temp_shp")
states_gdf = gpd.read_file("temp_shp/us_states.shp")


# Display the shapefile
my_map = Map(center=[37.0902, -95.7129], zoom=4)
my_map.add_vector(states_gdf, name="US States")

my_map


In [ ]:
## Add Vector
from spatialtsp import Map

# Initialize the map
my_map = Map(center=[37.0902, -95.7129], zoom=4)

# GeoJSON URLs
city_geojson_url = "https://github.com/opengeos/datasets/releases/download/us/us_cities.geojson"
county_geojson_url = "https://github.com/opengeos/datasets/releases/download/us/us_counties.geojson"

# Shapefile URL
states_shape_url = 'https://github.com/opengeos/datasets/releases/download/us/us_states.zip'

# Load GeoJSONs as GeoDataFrames
gdf_cities = gpd.read_file(city_geojson_url)
gdf_counties = gpd.read_file(county_geojson_url)

# Use add_vector to add GeoDataFrames to the map
my_map.add_vector(gdf_counties, name="US Counties")
my_map.add_vector(gdf_cities, name="US Cities")
# For the shapefile, directly use the URL
my_map.add_vector(states_shape_url, name="US States")

# Display the map
my_map

##Raster Map

In [ ]:
#change path
import os
os.chdir('D:/GIS program/Github/spatialtsp/spatialtsp-5')

In [ ]:
# import functions
!pip install localtileserver
import spatialtsp
from spatialtsp import Map

In [ ]:
# check the path of functions
from importlib import reload
reload(spatialtsp)

from spatialtsp import Map
m = Map()
print(dir(m)) 

import spatialtsp
print(spatialtsp.__file__)

In [ ]:
## overlay climate image(png)
from ipyleaflet import Map, ImageOverlay

m = Map(center=(25, -115), zoom=4)

image = ImageOverlay(
    url="https://i.imgur.com/06Q1fSz.png",
    # url='../06Q1fSz.png',
    bounds=((13, -130), (32, -100)),
)

m.add(image)
m

In [ ]:
## Overlay GIF of TSP into basemap
m = Map(center=(40, -100), zoom=4)

url = "https://graphdeeplearning.github.io/project/combinatorial-optimization/tsp-gif.gif"

# url='../06Q1fSz.png',
bounds = ((25, -125), (51, -66))
m.add_image(url, bounds)
m.add_layers_control()
# m.scroll_wheel_zoom = True
m

In [ ]:
## Overlay COloud Optimized GeoTiff into Basemap
from localtileserver import TileClient, get_leaflet_tile_layer, examples
from spatialtsp import Map

client = TileClient(
    "https://github.com/opengeos/datasets/releases/download/raster/srtm90.tif"
)

# Create ipyleaflet TileLayer from that server
t = get_leaflet_tile_layer(client)
# Create ipyleaflet map, add tile layer, and display
m = Map()
m.add(t)
m.center = client.center()
m.zoom = client.default_zoom
m